ich probiere mal aus, ob ich da auch loops einbauen könnte, um nicht so viele Wiederholungen zu haben

In [1]:
import camelot
import tabula
import pandas as pd

In [2]:
Firmen_2015 = camelot.read_pdf("Firmen_2015.pdf", flavor='stream', pages='all', table_areas=['0,842,595,0'])

In [12]:
#zurechtschneiden
test_df = Firmen_2015[0].df
Firmen_15 = test_df[12:]
firmen_2015 = Firmen_15.head(6)

#headers setzen
headers = ['Weglassen', 'Steuerbarer Gewinn ab CHF', 'weglassen', 'etwas seltsames',
           'lassweg', 'überflüssig']
firmen_2015.columns = headers

# aus einer Kolonne zwei Kolonnen machen
# von https://stackoverflow.com/questions/14745022/how-to-split-a-column-into-two-columns
getrennte_kolonnen15 = firmen_2015['etwas seltsames'].str.split(' ', 1, expand=True)

# überflüssige kolonnen löschen
del firmen_2015['Weglassen']
del firmen_2015['weglassen']
del firmen_2015['lassweg']
del firmen_2015['etwas seltsames']
del firmen_2015['überflüssig']

#die beiden df firmen_2015 und getrennte_kolonnen zusammenfügen
Firmen_2015_neu = pd.concat([firmen_2015, getrennte_kolonnen15], axis=1)

#headers setzen
headers = ['Steuerbarer Gewinn ab CHF', 'Anzahl Steuerpflichtige 2015', 'Betrag Kantonssteuer 2015 in CHF']
Firmen_2015_neu.columns = headers

#putzen
def geputzt(zeile):
    zeile = zeile.replace('(', '')
    zeile = zeile.replace(')', '')
    zeile = zeile.replace('\n', '') #das neu rein
    zeile = zeile.replace("'", "").strip()
    return zeile
Firmen_2015_neu['Steuerbarer Gewinn ab CHF'] = Firmen_2015_neu['Steuerbarer Gewinn ab CHF'].apply(geputzt)
Firmen_2015_neu['Anzahl Steuerpflichtige 2015'] = Firmen_2015_neu['Anzahl Steuerpflichtige 2015'].apply(geputzt)
Firmen_2015_neu['Betrag Kantonssteuer 2015 in CHF'] = Firmen_2015_neu['Betrag Kantonssteuer 2015 in CHF'].apply(geputzt)

#um 90 Grad drehen
firmen_15_neu = Firmen_2015_neu.T

# erste Zeile weg
Firmen15 = firmen_15_neu[1:]

# headers neu
headers = ['Steuerbarer Gewinn 0-10000 CHF', 'Steuerbarer Gewinn 10100-50000 CHF', 'Steuerbarer Gewinn 50100-250000 CHF', 
           'Steuerbarer Gewinn 250100 bis 1000000 CHF', 'Steuerbarer Gewinn ab 1000100 CHF', 'Total Steuerbarer Gewinn']
Firmen15.columns = headers

# Objekte in Integers verwandeln, damit ich die Zahlen addieren kann
Firmen15 = Firmen15.astype(int)

Firmen15.dtypes


Steuerbarer Gewinn 0-10000 CHF               int32
Steuerbarer Gewinn 10100-50000 CHF           int32
Steuerbarer Gewinn 50100-250000 CHF          int32
Steuerbarer Gewinn 250100 bis 1000000 CHF    int32
Steuerbarer Gewinn ab 1000100 CHF            int32
Total Steuerbarer Gewinn                     int32
dtype: object

In [ ]:

for cols in Firmen15:
    cols = Firmen15.filter(like = 'Steuerbarer Gewinn 0-10000 CHF')columns.tolist()
    Firmen15[cols] = Firmen15[cols].astype(int)

Firmen15.dtypes



cols = df.filter(like='AnalyzeParameters').columns.tolist()
df[cols] = df[cols].astype(np.int64)

In [ ]:
# Objekte in Integers verwandeln, damit ich die Zahlen addieren kann
Firmen15['Steuerbarer Gewinn 0-10000 CHF'] = Firmen15['Steuerbarer Gewinn 0-10000 CHF'].astype(str).astype(int)
Firmen15['Steuerbarer Gewinn 10100-50000 CHF'] = Firmen15['Steuerbarer Gewinn 10100-50000 CHF'].astype(str).astype(int)
Firmen15['Steuerbarer Gewinn 50100-250000 CHF'] = Firmen15['Steuerbarer Gewinn 50100-250000 CHF'].astype(str).astype(int)
Firmen15['Steuerbarer Gewinn 250100 bis 1000000 CHF'] = Firmen15['Steuerbarer Gewinn 250100 bis 1000000 CHF'].astype(str).astype(int)
Firmen15['Steuerbarer Gewinn ab 1000100 CHF'] = Firmen15['Steuerbarer Gewinn ab 1000100 CHF'].astype(str).astype(int)
Firmen15['Total Steuerbarer Gewinn'] = Firmen15['Total Steuerbarer Gewinn'].astype(str).astype(int)

# die Spalten addieren
Firmen15['Steuerbarer Gewinn ab 250100 CHF'] = Firmen15['Steuerbarer Gewinn 250100 bis 1000000 CHF'] + Firmen15['Steuerbarer Gewinn ab 1000100 CHF']

# die überflüssigen addierten Spalten raus
del Firmen15['Steuerbarer Gewinn 250100 bis 1000000 CHF']
del Firmen15['Steuerbarer Gewinn ab 1000100 CHF']

# die Reihenfolge der Kolonnen anpassen
Firmen15 = Firmen15[['Steuerbarer Gewinn 0-10000 CHF', 'Steuerbarer Gewinn 10100-50000 CHF', 
                     'Steuerbarer Gewinn 50100-250000 CHF', 'Steuerbarer Gewinn ab 250100 CHF', 
                     'Total Steuerbarer Gewinn']]

# den Betrag Kantonssteuer raus ziehen
Betrag_Kantonssteuer_2015 = Firmen15.drop(['Anzahl Steuerpflichtige 2015'])
Betrag_Kantonssteuer_2015